### Import libraries

In [ ]:
# Fix randomness and hide warnings
SEED = 76998669

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings as wr
wr.simplefilter(action='ignore', category=FutureWarning)
wr.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(SEED)

import logging

import random
random.seed(SEED)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
print(tf.__version__)

In [ ]:
# Import other libraries
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Load and process the dataset

In [ ]:
!wget https://github.com/GioBar00/AN2DL_23-24_Apple_Elixir_dataset/blob/46007b747f81ae768377893548949e31135fbc34/public_data_clean.npz

# Load dataset
dataset = np.load('public_data_clean.npz', allow_pickle=True)


In [ ]:
KEYS = list(dataset.keys())
IMAGES = dataset[KEYS[0]]
LABELS = dataset[KEYS[1]]

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# VARIABLES
indices_meme = []
images = []
labels = []

# CONFIGURATION
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

wr.simplefilter(action="ignore", category=FutureWarning)
wr.simplefilter(action="ignore", category=Warning)

#np.random.seed(SEED)
#rnd.seed(SEED)


# FUNCTIONS
def plot_losses(history, figsize):
  plt.figure(figsize=figsize)
  plt.plot(history['loss'], label='Training Loss', alpha=.8)
  plt.plot(history['val_loss'], label='Validation Loss', alpha=.8)
  plt.title('Loss')
  plt.legend()
  plt.grid(alpha=.3)

  plt.figure(figsize=figsize)
  plt.plot(history['accuracy'], label='Training Accuracy', alpha=.8)
  plt.plot(history['val_accuracy'], label='Validation Accuracy', alpha=.8)
  plt.title('Accuracy')
  plt.legend()
  plt.grid(alpha=.3)

  plt.show()

def apple_elixir_model(input_shape, output_shape, seed=SEED):

  tf.random.set_seed(seed)
  # Build the neural network layer by layer
  input_layer = tfkl.Input(shape=input_shape, name='Input')

  x = tfkl.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',name='conv00')(input_layer)
  x = tfkl.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',name='conv01')(x)
  x = tfkl.MaxPooling2D(name='mp0')(x)

  x = tfkl.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu',name='conv10')(x)
  x = tfkl.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu',name='conv11')(x)
  x = tfkl.MaxPooling2D(name='mp1')(x)

  x = tfkl.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu',name='conv20')(x)
  x = tfkl.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu',name='conv21')(x)
  x = tfkl.MaxPooling2D(name='mp2')(x)

  x = tfkl.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu',name='conv30')(x)
  x = tfkl.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu',name='conv31')(x)
  x = tfkl.GlobalAveragePooling2D(name='gap')(x)

  
  x = tfkl.Dense(units = 30, activation='relu')(x)
  x = tfkl.Dense(units = 20, activation='relu')(x)

  output_layer = tfkl.Dense(units=output_shape ,activation='softmax',name='Output')(x)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=input_layer, outputs=output_layer, name='Convnet')

  # Compile the model
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(weight_decay=5e-4), metrics=['accuracy'])

  return model

# Binary Classification Problem -> Sigmoid instead of Softmax for the Output, but you need to change the way you assign the value


# images = np.array(images).reshape(5004, 96*96*3)
images = np.array(images)
labels = np.array(labels)
labels_num = list(map(lambda x: [0,1] if x == "healty" else [1,0], labels))
labels_num = np.array(labels_num)

X_train_val, X_test, Y_train_val, Y_test = train_test_split(images, labels_num, test_size=500, random_state=SEED, stratify=labels_num)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=500, random_state=SEED, stratify=Y_train_val)

print('Training Set Shape: ', X_train.shape, Y_train.shape)
print('Validation Set Shape: ', X_val.shape, Y_val.shape)
print('Test Set Shape: ', X_test.shape, Y_test.shape)
print()

# Normalise
X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

input_shape = X_train.shape[1:]
output_shape = Y_train[1:]
print(f'Input shape of the Network {input_shape}')
print()


batch_size = 32
epochs = 500

model = apple_elixir_model(input_shape=input_shape,output_shape = output_shape)
model.summary()

history = model.fit(
  x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
  y = Y_train,
  batch_size = 16,
  epochs = 200,
  validation_data = (X_val, Y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
  callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history



plot_losses(history=history, figsize=(20,2))